In [1]:
!pip install robobrowser

In [2]:
# Python 2-3 compatibility
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
from builtins import *

In [3]:
import requests
from getpass import getpass
from time import sleep

In [4]:
import tinder_api as api
import fb_auth_token as fb_auth
import config

In [5]:
print('Enter your Facebook email and Password!')
email = input("Email:")
password = getpass("Password:")
#email = "rjbutler.throwaway1@gmail.com"
#password = "throwaway1337"

fb_token = config.fb_access_token = fb_auth.get_fb_access_token(email, password)
del password
fb_id = config.fb_user_id = fb_auth.get_fb_id(fb_token)
tinder_token = api.get_auth_token(fb_token, fb_id)

Enter your Facebook email and Password!
Email:ilsagruber97@gmail.com
Password:········
You have been successfully authorized!


In [6]:
def get_name(recs_result):
    return recs_result['user']['name'].lower()

In [10]:
def find_people(names, num_to_search, delay=0.5):
    """Find people on tinder by name
    Args:
        names: A list of strings to search for
        num_to_search: The minimum number of people to search through
        delay: The time in seconds between each request to tinder's servers.
        
    Returns: A tuple with a list of python dicts for each user that matched a name in `names`,
        and the number of people searched.
    """
    if isinstance(names, str):
        raise ValueError("`names` should be an iterable!")
    names = set(n.lower() for n in names)
    count = 0
    results = []
    while count < num_to_search:
        response = requests.get(config.host+'/v2/recs/core?locale=en-US', headers=api.headers)
        if not response.ok:
            break
        recs = response.json()["data"]["results"]
        
        for rec in recs:
            rec_name = get_name(rec)
            if rec_name in names:
                results.append(rec)
        count += len(recs)
        sleep(delay)
    return results, count

In [13]:
results = find_people(['RJ'], 100)
results

([], 132)